![dsl logo](https://github.com/BrockDSL/ARCH_Data_Explore/blob/main/dsl_logo.png?raw=true)

# A notebook for displaying the sentiment scores of various municipalities

In [ ]:
import pandas as pd

result = pd.read_csv('https://brockau.s3.us-east-2.amazonaws.com/sentiment_results.gz').set_index('domain')
result.crawl_date = pd.to_datetime(result.crawl_date)
result_columns = ['vader_comp','text_blob','rnn_fast_avg','bert_avg']

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt

def plot_sentiment(domain,algorithm,year):
    if year == 2020:
      year_data = result[result.crawl_date < '2021-01-01']
    else:
      year_data = result[result.crawl_date >= '2021-01-01']

    domain_data = year_data.loc[domain][[algorithm,'crawl_date']].set_index('crawl_date').groupby(pd.Grouper(freq='1M')).mean()
    fig, ax = plt.subplots(figsize=(12,8))
    x_pos = range(len(domain_data))
    ax.plot(x_pos,domain_data[algorithm])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(domain_data.index.strftime('%Y-%m-%d'), rotation=90)
    ax.set_ylabel('Sentiment Scores')
    ax.set_title(f"{domain} Sentiment Scores")

    #Plot a horizontal line showing mean over entire period
    total_mean = year_data.loc[domain][algorithm].mean()
    tm_line = ax.plot(x_pos, [total_mean]*len(x_pos), label="Total Period Mean")
    ax.legend()

interact(plot_sentiment, domain=set(result.index), algorithm=result_columns, year=[2020,2021])

interactive(children=(Dropdown(description='domain', options=('notl.com', 'portcolborne.ca', 'thorold.ca', 'pe…

<function __main__.plot_sentiment(domain, algorithm, year)>

In [ ]:
def plot_mean_sentiment(algorithm, year):
    if year == 2020:
      year_data = result[result.crawl_date < '2021-01-01']
    else:
      year_data = result[result.crawl_date >= '2021-01-01']

    barchart_data = year_data[algorithm].groupby('domain').mean().sort_values()
    fig, ax = plt.subplots(figsize=(12,8))
    x_pos = range(len(barchart_data))
    ax.bar(x_pos,barchart_data)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(barchart_data.index.values, rotation=90)
    ax.set_ylabel('Mean Sentiment Scores')
    ax.set_xlabel('Domains')
    ax.set_title(f'Mean Sentiment Scores by Domain for {year}')
    
interact(plot_mean_sentiment, algorithm=result_columns, year=[2020,2021])

interactive(children=(Dropdown(description='algorithm', options=('vader_comp', 'text_blob', 'rnn_fast_avg', 'b…

<function __main__.plot_mean_sentiment(algorithm, year)>